In [ ]:
!pip install python-Levenshtein
!pip install fitz
!pip install Levenshtein
!pip install PyMuPDF

In [ ]:
import pandas as pd
import json
import string
from sklearn.model_selection import train_test_split
from Levenshtein import distance as lev
import fitz
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def analyze_file_sent(name, phrase):

  doc = fitz.open(name)
  print(len(doc))

  phrase_split = phrase.split()
  l = len(phrase)
  page_count = 0
  info = []

  for page in doc:
    page_count+=1

    text = sent_tokenize(page.get_text(), language='russian')
    print(text[:10])
    for sent in text:

      if sent in phrase_split:
        area = page.search_for(sent)
        page.add_highlight_annot(sent)
      else:
        break
        dist = lev(sent,phrase)
        # print(dist/l, end=' ')
        if dist/l < 0.7:
          print()
        if dist>100000:
          if len(word) == len(word2):
            descr = 'Изменен символ: ' + word + ' and ' + word2
          else:
            descr = 'Разная длина слов: ' + word + ' and ' + word2

          info.append(
              {'filename': name.split('/')[-1], 'page': page_count, 'match': jaccard(word, word2), 'lev': lev(word,word2), 'subject':word, 'description': descr}
              )
          area = page.search_for(word)
          page.add_highlight_annot(area)


  doc.save("highlighted_text.pdf")


def process_mask(row_mask, t=4):
  '''
  Убирает все последовательности единиц короче <t>
  Если есть один нуль между единицами, кастует его к единице
  '''

  n = len(row_mask)
  post_mask = [0]*(n)
  hold = t # длина условного эталона - 3
  c=0

  for i in range(n):
    if row_mask[i]:
      c+=1
    else:
      if c >= hold:
        for j in range(i,i-c,-1): post_mask[j]=1
        c = 0

  for i in range(1,n-1):
    if post_mask[i]==0 and post_mask[i-1]==1 and post_mask[i+1]==1:
      post_mask[i]=1

  return post_mask


def jaccard(list1, list2):
  list1, list2 = list(list1), list(list2)
  intersection = len([x1==x2 for (x1,x2) in zip(list1,list2)])
  union = (len(list1) + len(list2))
  return float(intersection)/union

def split_s(s):
  return [char for char in s]


def analyze_file(name, phrase):
  doc = fitz.open(name)
  phrase_split = phrase.split()
  l = len(phrase)
  page_count = 0
  info = []

  for page in doc:
    page_count+=1

    text = page.get_text().split()
    text = [word for word in text if len(word)>3]

    mask = []

    for word in text:
      len_w = len(word)

      if word in phrase_split:
        mask.append(1)
      else:
        flag = False
        for word2 in phrase_split:
          if (len(word2)-len(word)) > 7:
            continue
          else:
            dist = lev(word,word2)/len_w
            if dist<0.2:
              if len(word) == len(word2): descr = 'Изменен символ: ' + word + ' and ' + word2
              else: descr = 'Разная длина слов: ' + word + ' and ' + word2

              info.append({'filename': name.split('/')[-1], 'page': page_count, 'match': jaccard(word, word2), 'lev': lev(word,word2), 'subject':word, 'description': descr})
              mask.append(1)
              flag = True
              break
          if flag:
            break

        if not(flag):
          mask.append(0)

    mask = process_mask(mask, 4)

    for r, word in zip(mask, text):
      if r: page.add_highlight_annot(page.search_for(word))


  doc.save("highlighted_text.pdf")
  return info

def make_csv(info):
  df = pd.DataFrame(info, columns=['filename','page','match','lev','subject','description'])
  df.to_csv('info.csv')

def main(name,phrase):

  info = analyze_file(name, phrase)
  make_csv(info)

  return info


In [ ]:
name = './example.pdf'
phrase = 'Слова Words'

info = main(name, phrase)
make_csv(info)

In [ ]:
len(info)

93

In [ ]:
df = pd.read_csv('./info.csv')

In [ ]:
df.head()

,Unnamed: 0,filename,page,match,lev,subject,description
0,0,Раздел ПД N 1. Пояснительная записка Том 5.7-1...,4,0.481481,1,«Реконструкция,Разная длина слов: «Реконструкция and Реконстр...
1,1,Раздел ПД N 1. Пояснительная записка Том 5.7-1...,4,0.461538,1,проект,Разная длина слов: проект and проекта
2,2,Раздел ПД N 1. Пояснительная записка Том 5.7-1...,4,0.500000,1,участке,Изменен символ: участке and участка
3,3,Раздел ПД N 1. Пояснительная записка Том 5.7-1...,4,0.500000,1,"пути»,","Изменен символ: пути», and пути»."
4,4,Раздел ПД N 1. Пояснительная записка Том 5.7-1...,4,0.500000,3,«Волго-Балтийское,Изменен символ: «Волго-Балтийское and Волго-Ба...


# токенизация чистка

In [ ]:
from Levenshtein import distance as lev

def jaccard(list1, list2):
 intersection = len(list(set(list1).intersection(list2)))
 union = (len(list1) + len(list2)) - intersection
 return float(intersection) / union

def split_s(s):
  return [char for char in s]

def string_classification(s1,s2):
  info = []
  if s1==s2:
    return 'ok'

  common_len_1 = len(s1)
  common_len_2 = len(s2)

  info.append(('Lenenstain dist: ', lev(s1,s2)/common_len_1))

  split_1 = s1.split()
  split_2 = s2.split()
  info.append(('Jaccar dist: ',1-jaccard(split_1,split_2)))

  split_len_1 = len(split_1)
  split_len_2 = len(split_2)

  count_perfect_words=0
  count_almost_words=0
  count_extra_words=0

  for word in split_1:

    if word in split_2:
      count_perfect_words+=1
    else:
      dist = len(word)
      almost_word = ''
      for word2 in split_2:
        curr_dist = lev(word,word2)
        if curr_dist < dist:
          dist = curr_dist
          almost_word = word2


      if dist < len(word2)/2 and almost_word!='':
        count_almost_words+=1
        info.append(('Similar word: '+ word +' and '+almost_word))
      else:
        count_extra_words+=1
        info.append(('Not found similar word: ', word))

  for word in split_2:
    if word in split_1:
      count_perfect_words+=1
    else:
      dist = len(word)
      almost_word = ''
      for word2 in split_1:
        curr_dist = lev(word,word2)
        if curr_dist < dist:
          dist = curr_dist
          almost_word = word2

      if dist < len(word2)/2 and almost_word!='':
        count_almost_words+=1
        info.append(('Similar word: '+ word +' and '+almost_word))
      else:
        count_extra_words+=1
        info.append(('Not found similar word: ', word))

  return info

In [ ]:
s =  'Строительство и что обустройство скважин куста №34 г.Москва'
s1 = 'Строительство и обустройство скважин куста №34 г.Москва положение' # лишнее слово
s2 = 'Строительство и обустройство скважин кста №34 г.Москва' # пропушен символ
s3 = 'Строительство и обустройство скважин №34 г.Москва' # пропушено слово
s4 = 'Строительство и обqстройство сквожин куста №34 г.Москва' # замен символ

In [ ]:
string_classification(s,s1)

[('Lenenstain dist: ', 0.23728813559322035),
 ('Jaccar dist: ', 0.2222222222222222),
 ('Not found similar word: ', 'что'),
 ('Not found similar word: ', 'положение')]

In [ ]:
string_classification(s,s2)

[('Lenenstain dist: ', 0.0847457627118644),
 ('Jaccar dist: ', 0.33333333333333337),
 ('Not found similar word: ', 'что'),
 'Similar word: куста and кста',
 'Similar word: кста and куста']

In [ ]:
string_classification(s,s3)

[('Lenenstain dist: ', 0.1694915254237288),
 ('Jaccar dist: ', 0.25),
 ('Not found similar word: ', 'что'),
 ('Not found similar word: ', 'куста')]

In [ ]:
string_classification(s,s4)

[('Lenenstain dist: ', 0.1016949152542373),
 ('Jaccar dist: ', 0.5),
 ('Not found similar word: ', 'что'),
 'Similar word: обустройство and обqстройство',
 'Similar word: скважин and сквожин',
 'Similar word: обqстройство and обустройство',
 'Similar word: сквожин and скважин']

In [ ]:
similarity_test

['«Разработка и реализация комплексного проекта реконструк-\nции Волго-Балтийского водного пути»  \nЭтап. Модернизация средств навигационной обстановки. \nII пусковой комплекс (ПК-2) - Модернизация средств навигаци-\nонной обстановки – маяки Бугровский, Стороженский, Свир-\nский, Малый, Хейнялуото \n ',
 '«Разработка и реализация комплексного проекта \nреконструкции Волго-Балтийского водного пути». \nЭтап. Модернизация средств навигационной обстановки. \nII пусковой комплекс (ПК-2) - Модернизация средств \nнавигационной обстановки - маяки Бугровский, \nСтороженский, Свирский ',
 'Основанием для разработки проектной документации по объекту «Разработка и \nреализация комплексного проекта реконструкции Волго-Балтийского водного пути». Этап. \n«Модернизация средств навигационной обстановки. II пусковой комплекс (ПК-2) - \nМодернизация средств навигационной обстановки - маяки Бугровский,Стороженский, \nСвирсккий   является подпрограмма «Морской и речной транспорт» Государственной  ',
 'СП 4

In [ ]:
clean_text('ПРОЕКТЫ ПО РЕКОНСТР УКЦИИ РЕМОНТОНЕПРИГО ДНЫХ МОСТОВ (СТРОИТЕЛЬСТВО И РЕК ОНСТРУКЦИЯ МОСТОВ И ПУ ТЕПРОВОДОВ)')

['проекты',
 'реконстр',
 'укции',
 'ремонтонеприго',
 'дных',
 'мостов',
 'строительство',
 'рек',
 'онструкция',
 'мостов',
 'тепроводов',
 'реконструкция',
 'моста',
 'через',
 'реку',
 'фарс',
 'автомобильной',
 'дороги',
 'кавказ']

In [ ]:
etalon = similarity_test[0]
for name in similarity_test:
  print(string_classification(etalon,name))

ok
[('Lenenstain dist: ', 0.10884353741496598), ('Jaccar dist: ', 0.475), 'Similar word: реконструк- and реконструкции', 'Similar word: ции and и', 'Similar word: пути» and пути».', ('Not found similar word: ', 'навигаци-'), ('Not found similar word: ', 'онной'), ('Not found similar word: ', '–'), ('Not found similar word: ', 'Свир-'), ('Not found similar word: ', 'ский,'), ('Not found similar word: ', 'Малый,'), ('Not found similar word: ', 'Хейнялуото'), 'Similar word: реконструкции and реконструк-', 'Similar word: пути». and пути»', 'Similar word: Свирский and Свир-']
[('Lenenstain dist: ', 0.46598639455782315), ('Jaccar dist: ', 0.6545454545454545), 'Similar word: реконструк- and реконструкции', 'Similar word: ции and и', 'Similar word: пути» and пути».', 'Similar word: навигаци- and реализация', 'Similar word: онной and речной', ('Not found similar word: ', '–'), 'Similar word: Бугровский, and пусковой', ('Not found similar word: ', 'Стороженский,'), 'Similar word: Свир- and и', '

In [ ]:
from Levenshtein import distance as lev

def find_abbreviation(name, text, use_only_letters=True, threshold=0.5):
  # clean text as list
  # not clean name as str ethelone
  name = clean_text(name)
  abb = create_abb(name, use_only_letters=use_only_letters)

  word_dist = []
  l = len(abb)

  for word in text:
    dist = lev(word,abb)/l
    word_dist.append([word, dist])

  word_dist.sort(key=lambda x: x[1])
  matches = [block[0] for block in word_dist if block[1]<threshold]
  return abb, matches


def create_abb(text, use_only_letters=True):
  abb=''
  for word in text:
    temp_char = word[0]
    if use_only_letters:
      if temp_char.isalpha():
        abb+=temp_char
    else:
      abb+=temp_char

  return abb

In [ ]:
s = ['рркпрцвпжфдыдл' ,'слова', 'уха', 'рррцввпэмснкмсноомх', 'ушел', 'муж', 'беда', 'ркпрцвпэмсноксномбссм', 'перед', 'этим', 'есть', 'аббреивиатру', 'найдем', 'ркпрцвпэмсноэксномбссм']
example = 'Разработка и реализация комплексного проекта реконструк-\nции Волго-Балтийского водного пути»  \nЭтап. Модернизация средств навигационной обстановки. \nII пусковой комплекс (ПК-2) - Модернизация средств навигаци-\nонной обстановки – маяки Бугровский, Стороженский, Свир-\nский, Малый, Хейнялуото \n '''

find_abbreviation(example, s)

('рркпрцввпэмснопкмсноомбсссмх',
 ['ркпрцвпэмсноксномбссм', 'ркпрцвпэмсноэксномбссм', 'рррцввпэмснкмсноомх'])

In [ ]:
cl_exmpl = clean_text(example) # list of words
abb = create_abb(cl_exmpl)
print(abb)
for name in s:
  print(lev(abb,name)/len(abb))

0.9285714285714286
0.9285714285714286
0.9642857142857143
0.7142857142857143
0.9285714285714286
1.0
0.32142857142857145
1.0
0.9642857142857143
0.9642857142857143
0.25
0.9642857142857143
0.9285714285714286
0.9642857142857143
0.9285714285714286
0.9285714285714286


In [ ]:
def process_mask(row_mask, t=5):
  '''
  Убирает все последовательности единиц короче <t>
  Если есть один нуль между единицами, кастует его к единице
  '''
  n = len(row_mask)
  post_mask = [0]*(n)
  hold = t # длина условного эталона - 3
  c=0

  for i in range(n):
    if row_mask[i]:
      c+=1
    else:
      if c >= hold:
        for j in range(i,i-c,-1): post_mask[j]=1
        c = 0

  for i in range(1,n-1):
    if post_mask[i]==0 and post_mask[i-1]==1 and post_mask[i+1]==1:
      post_mask[i]=1

  return post_mask



[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 